# Clustering
## Lecture objectives
1. Understand the purposes and potential uses of clustering
2. Learn more exploratory data analysis techniques, such as pairplots
3. Learn how to implement k-means clustering


## Why cluster?
Cluster analysis is an exploratory technique to identify sensible groupings in a dataset. The analyst has no prior knowledge of what these clusters are, and the data are not labeled with the "correct" cluster. Thus, cluster analysis is an *unsupervised* machine learning technique.

Some potential applications of clustering:
* Identify types of Marine Protected Area (e.g., [Bohorquez et al. 2019](https://www.sciencedirect.com/science/article/pii/S0308597X19304439))
* Identify types of street networks (e.g., [Barrington-Leigh and Millard-Ball 2020](https://www.pnas.org/content/117/4/1941))
* Identify types of neighborhood (e.g., [Kendig 2007](https://www.tandfonline.com/doi/abs/10.1080/01944367608977731))
* Identify types of transit agencies (e.g. [Ederer et al. 2019](https://journals.sagepub.com/doi/full/10.1177/0361198119852074))
* Identify patterns of cruising ror parking (e.g. [Millard-Ball, Weinberger & Hampshire 2021](https://findingspress.org/article/28061-the-shape-of-cruising))

Clustering the data in this way can help you see regularities in the data that you can then interpret. It might suggest policies that are appropriate for one group of cities or agencies but not another. Or it could identify a peer group against which to benchmark (say) affordable housing construction costs or transit on-time performance.

## Types of clustering
Formally, clustering takes a set of *N* objects and finds *K* groups based on a measure of similarity. For a technical yet accessible overview, I recommend [Jain 2010](https://www.sciencedirect.com/science/article/abs/pii/S0167865509002323). 

The two broad groups of clustering algorithms are *hierarchical* and *partitional*. 

Ederer et al. (2019), for example, use a hierarchical algorithm to classify transit agencies.

<img src="https://journals.sagepub.com/na101/home/literatum/publisher/sage/journals/content/trra/2019/trra_2673_11/0361198119852074/20191218/images/medium/10.1177_0361198119852074-fig1.gif" style="width:50%;"/>


But let's start with a partitional algorithm. The most popular is called *k-means*. Again, this is an exploratory data analysis process. The analyst needs to specify the number of clusters *K*, and should experiment with different values of *K* until a meaningful grouping emerges. Another way to choose *K* is the [elbow method](https://en.wikipedia.org/wiki/Elbow_method_(clustering)), but we won't discuss that here.

## Example: precinct-level voting
We'll use the `sklearn` library to implement the k-means algorithm. The aim: identify a typology of voters based on precinct-level data.

The California [Statewide Database](https://statewidedatabase.org), maintained by UC Berkeley, provides access to voting data. Your GitHub repository should include the November 2020 precinct data for Los Angeles County.

In [ ]:
import pandas as pd

df = pd.read_csv('data/c037_g20_sov_data_by_g20_srprec.csv')
df.head()

The unique identifier is given by the precinct column, `srprec`, so let's set that as the index.

In [ ]:
df.set_index('srprec', inplace=True)
df.index.is_unique  # verify that it's unique

What columns are in the data set?

In [ ]:
df.columns

There are obviously a lot of columns. You can see the [full codebook here](https://statewidedatabase.org/d10/g20.html). But the propositions are pretty self explanatory. `PRSDEM01` gives votes for Biden, `PRSREP01` for Trump, etc. Note the state Senate and Assembly races will be different depending on the precinct, so let's ignore those.

What do the data in these columns look like?

In [ ]:
df[['PRSDEM01','PRSREP01']].head()

We see that the numbers are in absolute terms. Let's convert them to vote share.

In [ ]:
# this is share of two-party vote (ignoring other candidates)
df['Biden_pc'] = df.PRSDEM01 / (df.PRSDEM01+df.PRSREP01)*100

Let's do the same for each proposition too. 

How can we get a list with the numbers of the propositions? We'll use a list comprehension to get the list of relevant columns.

In [ ]:
props = [col for col in df.columns if col.startswith('PR_') and col.endswith('Y')]
print(props)


And another list comprehension to get the relevant digit.

In [ ]:
# we can use our string indexing to get the 4th and 5th characters
# for example
print('PR_14_Y'[3:5])

# apply this in a list comprehension
props = [prop[3:5] for prop in props]
print(props)

Now, let's loop over this list of propositions to calculate the vote share.

In [ ]:
for prop in props:
    df[prop+'_pc_yes'] = df['PR_'+prop+'_Y'] / (df['PR_'+prop+'_Y'] 
                                              + df['PR_'+prop+'_N'])*100
df.head()

### Inspect and standardize the data

Number 1 rule: before applying any algorithm to your data, look at it!

We could create a scatterplot matrix manually. But `seaborn` has a [nice function to do this for us](https://seaborn.pydata.org/examples/scatterplot_matrix.html).

In [ ]:
import seaborn as sns

sns.pairplot?

We'll plot a subset of the columns, ignoring a few of the less critical propositions. Should stem cell research and dialysis rules really be on the ballot?

In [ ]:
# get a list of all the columns with 'pc' in the name
cols_to_plot = [col for col in df.columns if '_pc' in col]

# remove those we don't want
cols_to_plot.remove('14_pc_yes') 
cols_to_plot.remove('23_pc_yes') 
cols_to_plot.remove('24_pc_yes') 

# kind='reg' adds the line of best fit
ax = sns.pairplot(df[cols_to_plot], kind='reg')  

There are pretty strong relationships between Presidential voting and the propositions. All have a positive correlation except for Prop 20 (harsher sentencing) and Prop 22 (independent contractor status for drivers for Uber, Doordash, etc.). [A helpful reminder of the propositions is here](https://ballotpedia.org/California_2020_ballot_propositions).

But there isn't a perfect relationship. Perhaps cluster analysis can reveal some groupings?

First, it helps to pre-process the data in two ways:
* Let's align the data so that a higher percent means more progressive. This means using the percent "no" for Props 20 and 22
* We should standardize each variable to mean zero and standard deviation one. This helps ensure that the distances in multidimensional space are consistent. (Since we have a percentage measure, it won't make much difference compared to a variable like population, but it's good practice.)

In [ ]:
for prop in ['20','22']:
    df[prop+'_pc_no'] = 100 - df[prop+'_pc_yes']
    df.drop(columns=[prop+'_pc_yes'], inplace=True)

# then rerun the same code as above
cols_to_plot = [col for col in df.columns if '_pc' in col]
cols_to_plot.remove('14_pc_yes') 
cols_to_plot.remove('23_pc_yes') 
cols_to_plot.remove('24_pc_yes') 

# see https://scikit-learn.org/stable/modules/preprocessing.html for standardization
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(df[cols_to_plot])

# as in the previous lecture, 
# the scaler returns a numpy array, so we cast this as a DataFrame 
# and need to specify the column names and index
df_scaled = pd.DataFrame(scaler.transform(df[cols_to_plot]), 
                         columns=cols_to_plot, index=df.index)

Let's check that our data still look reasonable by rerunning the same pairplot.

Notice that the y axes run from about -3 to +3. This should be true for any standardized variable, as most observations are within 3 standard deviations of the mean.

In [ ]:
ax = sns.pairplot(df_scaled, kind='reg')

### KMeans in scikit-learn
The documention has some useful examples.

In [ ]:
from sklearn.cluster import KMeans
KMeans?

We can specify the number of clusters. Optionally, we can specify the random state, so that we can reproduce our work. 

Then we fit to our data, in this case `df_scaled`.

In [ ]:
kmeans = KMeans(n_clusters=5, random_state=0).fit(df_scaled)

Here, the error message is pretty helpful. Let's drop the NaNs.

In [ ]:
df_scaled = df_scaled.dropna()
kmeans = KMeans(n_clusters=5, random_state=0).fit(df_scaled)
print(kmeans)

It's not immediately obvious what we can do with this `KMeans` object. But let's explore it. Use the tab autocomplete to see the different methods.

In [ ]:
kmeans.

The cluster centers are defined by the (standardized) value for each variable. Each observation is assigned to the cluster with the closest center.

In [ ]:
kmeans.cluster_centers_

Notice that the `cluster_centers_` is an array that is `K x L`, where `K` is the number of clusters and `L` is the number of variables.

Here, we have 5 clusters, and we used 10 variables to define the clusters.

In [ ]:
print(kmeans.cluster_centers_.shape)
print(len(df_scaled.columns))

The `labels_` attribute gives the label of the cluster to which each observation (i.e., each precinct) is assigned.

In [ ]:
kmeans.labels_

So there are as many labels as rows in our dataframe.

In [ ]:
print(kmeans.labels_.shape)
print(len(df_scaled))

That means that we can simply add the cluster id back to our original dataframe!

In [ ]:
df_scaled['cluster_id'] = kmeans.labels_

How large is each cluster? Note that the algorithm doesn't aim to produce equal-size groupings.

In [ ]:
df_scaled.groupby('cluster_id').size()

### Visualizing the clusters
How best to visualize what the clusters mean? If we had just two columns, a scatterplot with a color code for each cluster would work well. But we have 13 dimensions.

One way is to redo our original scatter plot matrix, but with each cluster indicated.

In [ ]:
ax = sns.pairplot(df_scaled, hue='cluster_id', )

My preferred option, however, is a radar chart. Neither `seaborn` nor `matplotlib` do this natively, but [there is an example in the `matplotlib` gallery](https://matplotlib.org/stable/gallery/specialty_plots/radar_chart.html). I've just copied and pasted that code.

In [ ]:
# code from https://matplotlib.org/stable/gallery/specialty_plots/radar_chart.html

import numpy as np

import matplotlib.pyplot as plt
from matplotlib.patches import Circle, RegularPolygon
from matplotlib.path import Path
from matplotlib.projections.polar import PolarAxes
from matplotlib.projections import register_projection
from matplotlib.spines import Spine
from matplotlib.transforms import Affine2D


def radar_factory(num_vars, frame='circle'):
    """
    Create a radar chart with `num_vars` axes.

    This function creates a RadarAxes projection and registers it.

    Parameters
    ----------
    num_vars : int
        Number of variables for radar chart.
    frame : {'circle', 'polygon'}
        Shape of frame surrounding axes.

    """
    # calculate evenly-spaced axis angles
    theta = np.linspace(0, 2*np.pi, num_vars, endpoint=False)

    class RadarAxes(PolarAxes):

        name = 'radar'
        # use 1 line segment to connect specified points
        RESOLUTION = 1

        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
            # rotate plot such that the first axis is at the top
            self.set_theta_zero_location('N')

        def fill(self, *args, closed=True, **kwargs):
            """Override fill so that line is closed by default"""
            return super().fill(closed=closed, *args, **kwargs)

        def plot(self, *args, **kwargs):
            """Override plot so that line is closed by default"""
            lines = super().plot(*args, **kwargs)
            for line in lines:
                self._close_line(line)

        def _close_line(self, line):
            x, y = line.get_data()
            # FIXME: markers at x[0], y[0] get doubled-up
            if x[0] != x[-1]:
                x = np.append(x, x[0])
                y = np.append(y, y[0])
                line.set_data(x, y)

        def set_varlabels(self, labels):
            self.set_thetagrids(np.degrees(theta), labels)

        def _gen_axes_patch(self):
            # The Axes patch must be centered at (0.5, 0.5) and of radius 0.5
            # in axes coordinates.
            if frame == 'circle':
                return Circle((0.5, 0.5), 0.5)
            elif frame == 'polygon':
                return RegularPolygon((0.5, 0.5), num_vars,
                                      radius=.5, edgecolor="k")
            else:
                raise ValueError("Unknown value for 'frame': %s" % frame)

        def _gen_axes_spines(self):
            if frame == 'circle':
                return super()._gen_axes_spines()
            elif frame == 'polygon':
                # spine_type must be 'left'/'right'/'top'/'bottom'/'circle'.
                spine = Spine(axes=self,
                              spine_type='circle',
                              path=Path.unit_regular_polygon(num_vars))
                # unit_regular_polygon gives a polygon of radius 1 centered at
                # (0, 0) but we want a polygon of radius 0.5 centered at (0.5,
                # 0.5) in axes coordinates.
                spine.set_transform(Affine2D().scale(.5).translate(.5, .5)
                                    + self.transAxes)
                return {'polar': spine}
            else:
                raise ValueError("Unknown value for 'frame': %s" % frame)

    register_projection(RadarAxes)
    return theta


I adapted this example, putting it in a function called `radar_plot` that takes two arguments:
* the `kmeans` object
* the dataframe with the input data

In [ ]:
def radar_plot(kmeans, df_scaled):
    N  = kmeans.cluster_centers_.shape[1]  # number of columns / variables
    k = kmeans.n_clusters
    theta = radar_factory(N, frame='polygon')
    data = kmeans.cluster_centers_.T
    spoke_labels = [col for col in df_scaled.columns if col!='cluster_id']
    fig, ax = plt.subplots(figsize=(9, 9),
                                subplot_kw=dict(projection='radar'))
    fig.subplots_adjust(wspace=0.25, hspace=0.20, top=0.85, bottom=0.05)

    ax.plot(theta, data) #, color=color)
    ax.set_varlabels(spoke_labels)

    # add legend relative to top-left plot
    labels = ['Cluster {}'.format(kk) for kk in range(k)]
    ax.legend(labels, loc=(0.9, .95),
                                labelspacing=0.1, fontsize='small')

Let's call this function with our data.

In [ ]:
radar_plot(kmeans, df_scaled)

### Exploring different numbers of clusters
Here, the interesting finding is that all the clusters form concentric circles. There isn't a cluster of precincts that (say) votes against rent control but is progressive on the other items on the ballot.

We can certainly find these clusters if we increase `k`, but then these "weird" clusters have few precincts.

For example, let's try with `k=10`.

In [ ]:
# drop the old cluster id, so that we don't include it in our new estimates
df_scaled.drop(columns=['cluster_id'], inplace=True)  

# this is the same code as before
kmeans = KMeans(n_clusters=10, random_state=1).fit(df_scaled)
df_scaled['cluster_id'] = kmeans.labels_
print(df_scaled.groupby('cluster_id').size())
radar_plot(kmeans, df_scaled)

Let's go back to our original 5 clusters.

In [ ]:
df_scaled.drop(columns=['cluster_id'], inplace=True) 
kmeans = KMeans(n_clusters=5, random_state=1).fit(df_scaled)
df_scaled['cluster_id'] = kmeans.labels_
radar_plot(kmeans, df_scaled)

### Mapping the clusters
The Statewide Database team provide geographic boundary files as well as the vote counts. The shapefile for Los Angeles count is in your GitHub respository.

In [ ]:
import geopandas as gpd

gdf = gpd.read_file('data/srprec_037_g20_v01_shp/srprec_037_g20_v01.shp')
gdf.head()

Note that there is no projection file, so geopandas doesn't know the coordinate system
print(gdf.crs)

In [ ]:
print(gdf.crs)

The documentation online says it's in lat/lon, so let's set it to EPSG 4326.

In [ ]:
gdf.crs = 'EPSG:4326'

Before we do a join, let's look at the data to figure out the number of rows and the join column, and whether `srprec` is a unique identifiers.

In [ ]:
# looks like we can join on srprec, 
# but we'll need to set that as the index for gdf
print(df_scaled.head())
print(gdf.head())

In [ ]:
# we have more observations in our spatial data, so we can do a left join to that
# maybe some precincts have no voters?
print(len(gdf))
print(len(df_scaled))

In [ ]:
# both are unique, which makes things easier
print(df_scaled.index.is_unique)
print(gdf.SRPREC.is_unique)

In [ ]:
# do the join
gdf.set_index('SRPREC', inplace=True)
joinedGdf = gdf.join(df_scaled)
joinedGdf.head()

Let's map the clusters. We should color code by `cluster_id`.

In [ ]:
import contextily as ctx
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10,10))

joinedGdf.to_crs('EPSG:3857').plot('cluster_id', legend=True, ax = ax, alpha=0.4)
ctx.add_basemap(ax, zoom=12, source=ctx.providers.Stamen.TonerLite)

# drop Catalina Island
ax.set_ylim([3.98e6, 4.14e6])

# and we really don't need the axis ticks and labels, so we set them to an empty list
ax.set_xticks([])
ax.set_yticks([])

ax.set_title('Typology of voting, 2020 General Election', fontsize=20)

What can we do to improve the map?

The `source` keyword gives access to lots of options. Take a look at the possibilities with `ctx.providers`.

In [ ]:
ctx.providers

<div class="alert alert-block alert-info">
<strong>Exercise:</strong> How else would you improve the map?
</div>

There's no right answer here, but I first replace the missing data with an explicit "no data" label. To do that, we need to change the data type of `cluster_id` to string.

We can also remove the decimal point from the other cluster labels using the `str.replace()` function. We replace `.0` with an empty string.

In [ ]:
joinedGdf.cluster_id = joinedGdf.cluster_id.astype(str)
joinedGdf.cluster_id.fillna('No data', inplace=True)
joinedGdf.cluster_id = joinedGdf.cluster_id.str.replace('.0', '')
joinedGdf.cluster_id.head()

In the plot itself, we might:
* replace the colorbar with a legend. This is because we have discrete categories (0-5), not a continuous variable. That is done with the `categorical=True` keyword argument.
* add a legend title. We get the legend and then use the `set_title()` function.
* specify the colors. I find https://colorbrewer2.org the most helpful. 
* specify a gray for missing data (a grayscale color is a string between 0 and 1. E.g. 0 is black and 1 is white, with values in between representing progressively darker shades.

In [ ]:
# getting the colors into a colormap required some searching
# https://stackoverflow.com/questions/38882233/geopandas-matplotlib-plot-custom-colors
from matplotlib.colors import LinearSegmentedColormap
cmap = LinearSegmentedColormap.from_list(
    'mycmap', [(0, '#7fc97f'), (0.2, '#beaed4'), (0.4, '#fdc086'), 
               (0.6, '#ffff99'), (0.8, '#386cb0'), (1.0, '0.5')])

fig, ax = plt.subplots(figsize=(10,10))
joinedGdf.to_crs('EPSG:3857').plot('cluster_id', ax=ax, categorical=True, 
                                  legend=True, alpha=0.4,
                                  legend_kwds={'loc': 'upper left'})

# add a legend title
legend = ax.get_legend()
legend.set_title("Cluster", prop={'size':16} )

# all this is the same as before
ctx.add_basemap(ax, zoom=12, source=ctx.providers.Stamen.TonerLite)
ax.set_title('Typology of voting, 2020 General Election', fontsize=20)                           
ax.set_ylim([3.98e6, 4.14e6])
ax.set_xticks([])
ax.set_yticks([])

<div class="alert alert-block alert-info">
<h3>Key Takeaways</h3>
<ul>
  <li>Cluster analysis is an exploratory data tool</li>
  <li>Even if the clusters are pretty self-explanatory, they can be useful</li>
  <li>They are a general case of <em>data reduction</em>—reducing your data to something that can be readily intepreted</li>
  <li>They can be a starting point for further quantitative research—perhaps, use them as a variable in a regression model. </li>
  <li>They can also be useful for qualitative research. Perhaps you might do a case study of each cluster, picking the precinct/city/agency that is closest to each cluster center.</li>
</ul>
</div>